In [ ]:
#Generate Videos of Rays swimming based off of simulated swimming velocities
#Requires already having performed LBM simulation of a ray
#Disease Biophysics Group
#Written by John Zimmerman
#Updated 6/13/2024


import pandas as pd
import os
import numpy as np
from matplotlib.pyplot import *
import matplotlib
import GeoSwimmer
import SwimMesh

import scipy.stats
from scipy.interpolate import UnivariateSpline
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
def movAvg(x,N):
    return np.convolve(x, np.ones((N,))/N, mode='valid')

#Initial Variables
ReductionFactor = 1

#Time Steps
dt = 1.0/60.0 #Second per simulation step
dx = 0.1 #Size of each grid space
totalsimtime = 24.0 #seconds
wscale = 4 #worldscale

#Stimulation
Hz = 1.0 #Stimulation Frequency
cycletime = int(np.round(1 /(Hz*dt)))
print('Cycle frames: '+ str(cycletime))

#Mtf Variables - 
slantangle = -30.0 #Angle that the film is patterned at
maxr = 8  #Circle max - mm contraction radius assumed
minr = 1.4 #Circle min - mm contraction radius assumed

In [ ]:
#Contraction Profile
profile = np.loadtxt('Contraction_profile2.txt')
t = np.linspace(0,1,profile.size)
profilespline = UnivariateSpline(t,profile,k=3,s=.001)

#Transfer Profile
t = np.linspace(0,10,30)
a,b = 3,-0.2
g = scipy.stats.gamma(a,b).pdf(t)
g = g/g.max()
g = (1-g)*0.58
dif = g
difspline = UnivariateSpline(np.linspace(0,1,dif.size),dif,k=3,s=.001)

In [ ]:
swimname = 'GLIDAA'

#Path of where to save the video files to
savloc = 'G:\\...\\'

#Path of where the simulated travel veloctites are
velfiles = "G:\\Johnny Zimmerman\\DBG\\Data\\Simulated\\2023_07_06 SingleSim\\SwimVelocities\\"

#Load Velocity
velocities = np.loadtxt(velfiles+swimname+'_velocity.txt')

tri,vertdict = SwimMesh.generateMeshHalf(swimname,40,gscale=wscale,arg='pqa1.2')    
verts = np.array(tri['vertices'])
triangles = tri['triangles']
slantr = SwimMesh.reduce_slant(verts,slantangle)

savecheck = os.path.exists(savloc+swimname)
if savecheck == False:
    os.makedirs(savloc+swimname)

In [ ]:
pos = 0
i = 0
#num = 0
lw = 1
for vel in velocities[:]:
    pos = pos + vel*dt/dx
    xnew,ynew,znew = SwimMesh.solveContraction(verts,minr,maxr,slantangle,slantr,i%cycletime,cycletime,profilespline,difspline)
    
    ##Plot Velocity Top Down##
    fig, ax = subplots(figsize=(10,4))
    #ax.pcolormesh(xs,ys, M,cmap=cms,vmin=vlowlim,vmax=vuplim)

    for vertices in triangles:
        ax.plot(-xnew[vertices]+pos,ynew[vertices],'r--',linewidth=lw)
        ax.plot(-xnew[[vertices[-1],vertices[0]]]+pos,ynew[[vertices[-1],vertices[0]]],'r--',linewidth=lw)
        ax.plot(-xnew[vertices]+pos,-ynew[vertices],'r--',linewidth=lw)
        ax.plot(-xnew[[vertices[-1],vertices[0]]]+pos,-ynew[[vertices[-1],vertices[0]]],'r--',linewidth=lw)

    ax.set_xlabel('X (mm)',size=30,labelpad=20)
    ax.set_ylabel('Y (mm)',size=30,labelpad=20)
    
    xlim(-25,55)
    ylim(-10,10)
    #xlim(-25,55)
    #ylim(-10,10)
    
    savefig(savloc+swimname+'\\'+str(i)+'.png')
    clf()
    close()
    
    i+=1
    print(i)